In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("../new_data_archive/Housing.csv")
df.head(5)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7229300521,20141013T000000,231300.0,2,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


# 1) Initial Data Cleaning

**Handle the `bedrooms` outlier**

In [3]:
df = df[df['bedrooms'] < 10]

**Change `zipcode` to a string**

In [4]:
df['zipcode'] = df['zipcode'].astype(str)

# 2) Feature Engineering

**Handle the `date` column**
- Convert 'date' to a date time object
- Extract sale_year and sale_month
- delete the date column 

In [5]:
df['date'] = pd.to_datetime(df['date'])

df['sale_year'] = df['date'].dt.year
df['sale_month'] = df['date'].dt.month

df.head(5)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sale_year,sale_month
0,7229300521,2014-10-13,231300.0,2,1.00,1180,5650,1.0,0,0,...,0,1955,0,98178,47.5112,-122.257,1340,5650,2014,10
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0,0,...,400,1951,1991,98125,47.7210,-122.319,1690,7639,2014,12
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,0,0,...,0,1933,0,98028,47.7379,-122.233,2720,8062,2015,2
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0,0,...,910,1965,0,98136,47.5208,-122.393,1360,5000,2014,12
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0,0,...,0,1987,0,98074,47.6168,-122.045,1800,7503,2015,2


**Handle `yr_built` and `yr_renovated`:**
- Create a binary `was_renovated` feature.
- Create an `effective_year` feature (uses `yr_renovated` if it exists, else `yr_built`).
- Create `effective_age` (age of the house at the time of sale).

In [6]:
df['was_renovated'] = (df['yr_renovated'] > 0).astype(int)
df['effective_year'] = df.apply(lambda row: row['yr_renovated'] if row['yr_renovated'] > 0 else row['yr_built'], axis=1)
df['effective_age'] = df['sale_year'] - df['effective_year']
df.head(5)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,zipcode,lat,long,sqft_living15,sqft_lot15,sale_year,sale_month,was_renovated,effective_year,effective_age
0,7229300521,2014-10-13,231300.0,2,1.00,1180,5650,1.0,0,0,...,98178,47.5112,-122.257,1340,5650,2014,10,0,1955,59
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0,0,...,98125,47.7210,-122.319,1690,7639,2014,12,1,1991,23
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,0,0,...,98028,47.7379,-122.233,2720,8062,2015,2,0,1933,82
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0,0,...,98136,47.5208,-122.393,1360,5000,2014,12,0,1965,49
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0,0,...,98074,47.6168,-122.045,1800,7503,2015,2,0,1987,28


# 3) Transforming Skewed Features

**We need to apply a log transform to our right-skewed target and predictor variables to normalize them**


In [7]:
df['price_log'] = np.log1p(df['price'])

df['sqft_living_log'] = np.log1p(df['sqft_living'])
df['sqft_lot_log'] = np.log1p(df['sqft_lot'])
df['sqft_living15_log'] = np.log1p(df['sqft_living15'])
df['sqft_lot15_log'] = np.log1p(df['sqft_lot15'])
df.head(5)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sale_year,sale_month,was_renovated,effective_year,effective_age,price_log,sqft_living_log,sqft_lot_log,sqft_living15_log,sqft_lot15_log
0,7229300521,2014-10-13,231300.0,2,1.00,1180,5650,1.0,0,0,...,2014,10,0,1955,59,12.351475,7.074117,8.639588,7.201171,8.639588
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0,0,...,2014,12,1,1991,23,13.195616,7.852050,8.887791,7.433075,8.941153
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,0,0,...,2015,2,0,1933,82,12.100718,6.647688,9.210440,7.908755,8.995041
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0,0,...,2014,12,0,1965,49,13.311331,7.581210,8.517393,7.215975,8.517393
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0,0,...,2015,2,0,1987,28,13.142168,7.427144,8.997271,7.496097,8.923191


**Drop unnecessary columns**


In [8]:
cols_to_drop = [
    'price',            # Replaced by 'price_log'
    'date',             # Replaced by 'sale_year', 'sale_month'
    'sqft_above',       # Redundant with 'sqft_living'
    'sqft_basement',    # Redundant with 'sqft_living'
    'yr_built',         # Replaced by 'effective_age'
    'yr_renovated',     # Replaced by 'effective_age'
    'sale_year',        # Replaced by 'effective_age'
    'effective_year',   # Replaced by 'effective_age'
    'lat',              # We are using zipcode instead
    'long',             # We are using zipcode instead
    'sqft_living',      # Replaced by 'sqft_living_log'
    'sqft_lot',         # Replaced by 'sqft_lot_log'
    'sqft_living15',    # Replaced by 'sqft_living15_log'
    'sqft_lot15',        # Replaced by 'sqft_lot15_log (which is also dropped to prevent redundancy)'  
    'sqft_lot15_log'    # Removed to prevent redundancy because sqft_lot_log is the same as this one
]

df_processed = df.drop(columns=cols_to_drop)

In [9]:
df_processed.to_csv('processed_housing.csv', index=False)
print("Processed dataset saved as processed_housing.csv")

Processed dataset saved as processed_housing.csv
